Load SSD model

In [3]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)

print("Project root added to path:", PROJECT_ROOT)


import torch

from models.SSD.config import NUM_CLASSES
from models.SSD.models.ssd import SSD

DEVICE = torch.device("cpu")  # SSD evaluation on CPU for consistency

print("NUM_CLASSES =", NUM_CLASSES)
print("DEVICE =", DEVICE)

Project root added to path: /Users/prekshadahal/Desktop/Masters/Year 2/Computer Vision/Face-Mask Detection
NUM_CLASSES = 4
DEVICE = cpu


Run inference

In [4]:
model = SSD(num_classes=NUM_CLASSES)

model.load_state_dict(
    torch.load("../outputs/checkpoints/ssd_epoch_30.pth", map_location=DEVICE)
)

model.to(DEVICE)
model.eval()

print("✅ SSD checkpoint loaded and ready")

✅ SSD checkpoint loaded and ready


In [5]:
from torch.utils.data import DataLoader
from models.SSD.utils.dataset import MaskDataset
import pandas as pd

df = pd.read_csv("../data/processed/annotations.csv")

# Same split as YOLO
val_df = df.sample(frac=0.2, random_state=42)

val_dataset = MaskDataset(
    val_df,
    image_dir="../data/processed/images",
    label_map={
        "with_mask": 1,
        "without_mask": 2,
        "mask_weared_incorrect": 3
    }
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False
)

print("Validation samples:", len(val_dataset))

Validation samples: 700


In [6]:
all_preds = []
all_targets = []

with torch.no_grad():
    for images, gt_boxes, gt_labels in val_loader:
        images = images.to(DEVICE)

        cls_preds, box_preds = model(images)

        all_preds.append((cls_preds, box_preds))
        all_targets.append((gt_boxes, gt_labels))

print("Inference completed on validation set")

Inference completed on validation set


Decode boxes + apply softmax + confidence threshold + NMS

In [7]:
import torch
import torch.nn.functional as F
from torchvision.ops import nms

def postprocess_ssd(cls_preds, box_preds, score_thresh=0.5, iou_thresh=0.5):
    """
    Post-process SSD outputs for your current implementation
    """
    # Concatenate feature maps if multiple (your SSD might already return flattened)
    if isinstance(cls_preds, list):
        cls_preds = torch.cat([c.view(-1, NUM_CLASSES) for c in cls_preds], dim=0)
        box_preds = torch.cat([b.view(-1, 4) for b in box_preds], dim=0)
    else:
        cls_preds = cls_preds.view(-1, NUM_CLASSES)
        box_preds = box_preds.view(-1, 4)

    # Softmax probabilities
    scores = F.softmax(cls_preds, dim=1)

    # Remove background class (0)
    scores_bg_removed = scores[:, 1:]
    labels = scores_bg_removed.argmax(dim=1) + 1
    confidences = scores_bg_removed.max(dim=1).values

    # Confidence threshold
    mask = confidences > score_thresh
    boxes = box_preds[mask]
    labels = labels[mask]
    confidences = confidences[mask]

    # Apply NMS
    if boxes.shape[0] > 0:
        keep = nms(boxes, confidences, iou_thresh)
        boxes = boxes[keep]
        labels = labels[keep]
        confidences = confidences[keep]

    return boxes, labels, confidences

Compute IoU between predicted boxes and ground truth

In [8]:
def compute_iou(box1, box2):
    """Compute IoU between 2 boxes [x1,y1,x2,y2]"""
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter

    return inter / union if union > 0 else 0

Precision/Recall

In [9]:
TP, FP, FN = 0, 0, 0

for (cls_preds, box_preds), (gt_boxes, gt_labels) in zip(all_preds, all_targets):
    pred_boxes, pred_labels, pred_scores = postprocess_ssd(cls_preds, box_preds, score_thresh=0.5)
    
    gt_boxes = gt_boxes[0]  # batch_size=1
    gt_labels = gt_labels[0]

    matched = set()
    for pb, pl in zip(pred_boxes, pred_labels):
        found = False
        for i, (gb, gl) in enumerate(zip(gt_boxes, gt_labels)):
            if i in matched:
                continue
            if pl == gl and compute_iou(pb, gb) >= 0.5:
                TP += 1
                matched.add(i)
                found = True
                break
        if not found:
            FP += 1
    FN += len(gt_boxes) - len(matched)

precision = TP / (TP + FP + 1e-3)
recall = TP / (TP + FN + 1e-3)

print("SSD Precision:", precision)
print("SSD Recall:", recall)

SSD Precision: 0.0003831356444535702
SSD Recall: 0.13714266122476967


FPS Benchmark

In [10]:
import time

start = time.time()
with torch.no_grad():
    for images, _, _ in val_loader:
        images = images.to(DEVICE)
        _ = model(images)
end = time.time()

fps = len(val_loader) / (end - start)
print(f"SSD FPS: {fps:.2f}")

SSD FPS: 4.09
